<a href="https://colab.research.google.com/github/bradleyboehmke/uc-bana-4080/blob/main/example-notebooks/26_random_forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forests: Ensemble Power and Robustness

This notebook contains code examples from the **Random Forests: Ensemble Power and Robustness** chapter (Chapter 26) of the BANA 4080 textbook. Follow along to practice building and tuning random forest models using scikit-learn.

## 📚 Chapter Overview

Random forests apply the "wisdom of crowds" principle to machine learning by combining hundreds of decision trees trained on different samples of data. Through bootstrap sampling and feature randomness, random forests create diverse trees whose errors cancel out when averaged, resulting in models that are more accurate, stable, and robust than individual trees.

## 🎯 What You'll Practice

- Understand how bootstrap aggregating (bagging) creates diverse decision trees
- Implement feature randomness to decorrelate trees and improve performance
- Build classification and regression models using `RandomForestClassifier` and `RandomForestRegressor`
- Tune key hyperparameters (`n_estimators`, `max_features`, `max_depth`, `min_samples_split/leaf`)
- Compare single decision trees vs. bagged trees vs. random forests
- Apply random forests to real business problems

## 💡 How to Use This Notebook

1. **Read the chapter first** - This notebook supplements the textbook, not replaces it
2. **Run cells sequentially** - Code builds on previous examples
3. **Experiment freely** - Modify code to test your understanding
4. **Practice variations** - Try different parameters and datasets to reinforce learning

## Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, 
    classification_report,
    mean_squared_error,
    mean_absolute_error,
    r2_score
)
from sklearn.datasets import load_iris
from sklearn import tree

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Libraries imported successfully!")

## Tree Instability and the Ensemble Solution

Decision trees suffer from **high variance** - small changes in training data can produce dramatically different trees. Random forests solve this problem by combining multiple trees trained on different samples of data. Let's see this instability in action.

In [ ]:
# Load iris dataset for demonstration
iris = load_iris()
X_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
y_iris = pd.Series(iris.target)

print(f"Dataset loaded: {len(X_iris)} samples, {len(X_iris.columns)} features")
print(f"Classes: {', '.join(iris.target_names)}")

In [ ]:
# Demonstrate tree instability by training trees on different bootstrap samples
n_samples = len(X_iris)
n_trees = 3

print("Training 3 decision trees on different bootstrap samples...\n")

for i in range(n_trees):
    # Create bootstrap sample (random sample WITH replacement)
    bootstrap_indices = np.random.choice(n_samples, size=n_samples, replace=True)
    X_bootstrap = X_iris.iloc[bootstrap_indices]
    y_bootstrap = y_iris.iloc[bootstrap_indices]
    
    # Train decision tree on this bootstrap sample
    dt = DecisionTreeClassifier(max_depth=3, random_state=42)
    dt.fit(X_bootstrap, y_bootstrap)
    
    # Get the first split feature
    first_split_feature = X_iris.columns[dt.tree_.feature[0]]
    first_split_threshold = dt.tree_.threshold[0]
    
    print(f"Tree {i+1}:")
    print(f"  First split: {first_split_feature} <= {first_split_threshold:.2f}")
    print(f"  Training accuracy: {dt.score(X_bootstrap, y_bootstrap):.3f}")
    print(f"  Number of unique samples: {len(np.unique(bootstrap_indices))} out of {n_samples}")
    print()

**Key Observation**: Even though all three trees were trained on the same underlying dataset, they make different first splits because each saw a different bootstrap sample. This instability is a weakness of individual trees that random forests address through averaging.

### 🏃‍♂️ Try It Yourself

Modify the code above to:
1. Increase `n_trees` to 5 and observe how diverse the first splits become
2. Change `max_depth=3` to `max_depth=5` and see if trees become more or less similar
3. Set `replace=False` in the bootstrap sampling (creating subsamples instead) - how does this affect tree diversity?

In [ ]:
# Your experimental code here


### The Ensemble Solution: Combining Multiple Trees

The key insight of random forests: while individual trees are unstable and make errors, their **collective wisdom** tends to be more reliable. Let's compare a single tree to an ensemble of trees.

In [ ]:
# Split iris data for fair comparison
X_train, X_test, y_train, y_test = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42
)

# Train a single decision tree
single_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
single_tree.fit(X_train, y_train)

# Train a random forest (ensemble of trees)
random_forest = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
random_forest.fit(X_train, y_train)

# Compare performance
print("Performance Comparison:\n")
print(f"Single Decision Tree:")
print(f"  Training accuracy: {single_tree.score(X_train, y_train):.3f}")
print(f"  Test accuracy:     {single_tree.score(X_test, y_test):.3f}")
print()
print(f"Random Forest (100 trees):")
print(f"  Training accuracy: {random_forest.score(X_train, y_train):.3f}")
print(f"  Test accuracy:     {random_forest.score(X_test, y_test):.3f}")
print()
print(f"Improvement in test accuracy: {(random_forest.score(X_test, y_test) - single_tree.score(X_test, y_test)):.3f}")

**Why does this work?** When one tree makes an error, other trees are likely to get it right. By averaging predictions (or voting in classification), individual errors cancel out while the true signal reinforces across all trees.

### 🏃‍♂️ Try It Yourself

Experiment with the number of trees:
1. Try `n_estimators=10, 50, 100, 200, 500` and plot how test accuracy changes
2. Calculate the overfitting gap (training accuracy - test accuracy) for both models
3. Which model generalizes better? Why?

In [ ]:
# Your code here


## Bootstrap Aggregating (Bagging)

**Bagging** is the foundation of random forests. It combines **bootstrap sampling** (creating random samples with replacement) with **aggregation** (averaging predictions). Let's understand each component.

### Understanding Bootstrap Sampling

Bootstrap sampling creates new datasets by randomly selecting observations **with replacement**. This means:
- Some observations appear multiple times
- Some observations don't appear at all (~37% are left out)
- Each sample has the same size as the original

In [ ]:
# Demonstrate bootstrap sampling with a simple example
original_data = np.arange(1, 21)  # 20 observations numbered 1-20

print("Original data (20 observations):")
print(original_data)
print()

# Create 3 bootstrap samples
for i in range(3):
    bootstrap_sample = np.random.choice(original_data, size=len(original_data), replace=True)
    unique_values = np.unique(bootstrap_sample)
    missing_values = set(original_data) - set(unique_values)
    
    print(f"Bootstrap Sample {i+1}:")
    print(f"  Unique observations: {len(unique_values)} out of 20")
    print(f"  Missing observations: {len(missing_values)}")
    print(f"  Sample: {bootstrap_sample[:10]}... (showing first 10)")
    print()

**Key Insight**: Each bootstrap sample is different, which creates the diversity we need for random forests. About 63% of the original observations appear in each sample (some multiple times).

### Bagging in Practice: Regression Example

Let's build a simple regression example to see how bagging reduces prediction error by averaging multiple trees.

In [ ]:
# Create synthetic regression data (sin wave with noise)
np.random.seed(42)
X_sin = np.linspace(0, 4*np.pi, 100).reshape(-1, 1)
y_sin_true = np.sin(X_sin).ravel()
y_sin = y_sin_true + np.random.normal(0, 0.3, len(X_sin))  # Add noise

# Create test grid for smooth predictions
X_grid = np.linspace(0, 4*np.pi, 300).reshape(-1, 1)
y_grid_true = np.sin(X_grid).ravel()

print(f"Training data: {len(X_sin)} points")
print(f"Test grid: {len(X_grid)} points")

In [ ]:
# Train 10 trees on bootstrap samples and average their predictions
n_trees = 10
predictions = []

for i in range(n_trees):
    # Create bootstrap sample
    indices = np.random.choice(len(X_sin), size=len(X_sin), replace=True)
    X_bootstrap = X_sin[indices]
    y_bootstrap = y_sin[indices]
    
    # Train tree on bootstrap sample
    tree_model = DecisionTreeRegressor(max_depth=5, random_state=i)
    tree_model.fit(X_bootstrap, y_bootstrap)
    
    # Predict on test grid
    y_pred = tree_model.predict(X_grid)
    predictions.append(y_pred)

# Calculate bagged prediction (average of all trees)
bagged_prediction = np.mean(predictions, axis=0)

# Calculate errors
single_tree_mse = mean_squared_error(y_grid_true, predictions[0])
bagged_mse = mean_squared_error(y_grid_true, bagged_prediction)

print(f"Single tree MSE: {single_tree_mse:.4f}")
print(f"Bagged prediction MSE: {bagged_mse:.4f}")
print(f"Error reduction: {((single_tree_mse - bagged_mse) / single_tree_mse * 100):.1f}%")

**Why averaging helps**: Individual trees overfit to noise in their specific bootstrap sample. When we average predictions, the noise cancels out while the true signal reinforces!

### The Power of More Trees

As we add more trees, error typically decreases sharply at first, then levels off. Let's visualize this.

In [ ]:
# Track error as we add more trees
max_trees = 50
errors = []
all_predictions = []

for i in range(max_trees):
    # Create bootstrap sample and train tree
    indices = np.random.choice(len(X_sin), size=len(X_sin), replace=True)
    X_bootstrap = X_sin[indices]
    y_bootstrap = y_sin[indices]
    
    tree_model = DecisionTreeRegressor(max_depth=5, random_state=i)
    tree_model.fit(X_bootstrap, y_bootstrap)
    
    y_pred = tree_model.predict(X_grid)
    all_predictions.append(y_pred)
    
    # Calculate error using average of all trees so far
    avg_prediction = np.mean(all_predictions, axis=0)
    mse = mean_squared_error(y_grid_true, avg_prediction)
    errors.append(mse)

# Plot error vs number of trees
plt.figure(figsize=(8, 5))
plt.plot(range(1, max_trees + 1), errors, linewidth=2, color='darkblue')
plt.axhline(y=errors[-1], color='red', linestyle='--', alpha=0.5, 
            label=f'Final error with {max_trees} trees')
plt.xlabel('Number of Trees', fontsize=11)
plt.ylabel('Mean Squared Error', fontsize=11)
plt.title('Error Decreases as More Trees Are Added', fontsize=13, fontweight='bold')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print(f"Error with 1 tree:  {errors[0]:.4f}")
print(f"Error with 10 trees: {errors[9]:.4f} (reduction: {(1 - errors[9]/errors[0])*100:.1f}%)")
print(f"Error with 50 trees: {errors[49]:.4f} (reduction: {(1 - errors[49]/errors[0])*100:.1f}%)")

**Key Observation**: 
- Sharp decrease in error with the first 10-15 trees
- Diminishing returns after ~20 trees
- More trees = more diverse perspectives = better collective wisdom!

### 🏃‍♂️ Try It Yourself

Experiment with bagging:
1. Modify `max_depth` in the trees (try 3, 7, 10) - how does tree complexity affect bagging benefits?
2. Change the noise level in the data (modify `0.3` to `0.1` or `0.5`) - does bagging help more with noisier data?
3. Track how many unique training samples each bootstrap contains on average

In [ ]:
# Your experimental code here


## Feature Randomness: The Key Innovation

Bagging alone creates diversity through bootstrap sampling. But random forests add **one more crucial source of randomness**: at each split, only consider a random subset of features. This simple change dramatically improves performance.

### How Feature Randomness Works

For a dataset with *p* features, at each node:
1. Randomly select a subset of features (typically √p for classification, p/3 for regression)
2. Find the best split using **only** these randomly selected features
3. Repeat this at every node in every tree

This means different trees split on different features, making their predictions more independent.

In [ ]:
# Load a dataset to demonstrate feature randomness
# We'll use the iris dataset which has 4 features
from sklearn.datasets import load_iris

iris = load_iris()
X_iris_demo = iris.data
y_iris_demo = iris.target

n_features = X_iris_demo.shape[1]
print(f"Dataset has {n_features} features")
print(f"Feature names: {iris.feature_names}")
print()
print(f"For classification, typical max_features = √{n_features} = {int(np.sqrt(n_features))}")
print(f"For regression, typical max_features = {n_features}/3 = {n_features // 3}")

### Comparing Bagged Trees vs. Random Forests

Let's compare:
- **Bagged Trees**: Bootstrap sampling + ALL features at each split
- **Random Forest**: Bootstrap sampling + RANDOM SUBSET of features at each split

In [ ]:
# Manual implementation to show the difference
# We'll use a regression example for clearer error metrics

# Create regression dataset
from sklearn.datasets import make_regression
X_reg, y_reg = make_regression(n_samples=200, n_features=10, n_informative=8, 
                               noise=10, random_state=42)

# Split data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)

print(f"Training samples: {len(X_train_reg)}")
print(f"Features: {X_train_reg.shape[1]}")
print(f"\nWe'll compare bagging with all features vs. random forests with feature randomness")

In [ ]:
# Build bagged trees (all features at each split)
n_trees = 100
bagged_predictions = []

for i in range(n_trees):
    # Bootstrap sample
    indices = np.random.choice(len(X_train_reg), size=len(X_train_reg), replace=True)
    X_bootstrap = X_train_reg[indices]
    y_bootstrap = y_train_reg[indices]
    
    # Train tree with ALL features (max_features=None)
    tree = DecisionTreeRegressor(max_features=None, min_samples_split=10, 
                                 min_samples_leaf=4, random_state=i)
    tree.fit(X_bootstrap, y_bootstrap)
    
    # Predict
    bagged_predictions.append(tree.predict(X_test_reg))

# Average predictions
bagged_avg = np.mean(bagged_predictions, axis=0)
bagged_mse = mean_squared_error(y_test_reg, bagged_avg)

print(f"Bagged Trees (all features):")
print(f"  Test MSE: {bagged_mse:.2f}")

In [ ]:
# Build random forest (random subset of features at each split)
rf_predictions = []
max_features = X_train_reg.shape[1] // 3  # p/3 for regression

for i in range(n_trees):
    # Bootstrap sample
    indices = np.random.choice(len(X_train_reg), size=len(X_train_reg), replace=True)
    X_bootstrap = X_train_reg[indices]
    y_bootstrap = y_train_reg[indices]
    
    # Train tree with RANDOM SUBSET of features
    tree = DecisionTreeRegressor(max_features=max_features, min_samples_split=10,
                                 min_samples_leaf=4, random_state=i)
    tree.fit(X_bootstrap, y_bootstrap)
    
    # Predict
    rf_predictions.append(tree.predict(X_test_reg))

# Average predictions
rf_avg = np.mean(rf_predictions, axis=0)
rf_mse = mean_squared_error(y_test_reg, rf_avg)

print(f"Random Forest (max_features={max_features}):")
print(f"  Test MSE: {rf_mse:.2f}")
print()
print(f"Improvement from feature randomness: {((bagged_mse - rf_mse) / bagged_mse * 100):.1f}%")

**Why does feature randomness help?**

- **Decorrelates trees**: Forces trees to explore different feature combinations
- **Prevents feature dominance**: Strong features don't control every tree
- **Discovers hidden patterns**: Weaker features get a chance to contribute
- **More independent errors**: When averaged, errors cancel out more effectively

### Using Scikit-learn's Built-in Random Forest

In practice, we use scikit-learn's `RandomForestClassifier` and `RandomForestRegressor` which handle all the bootstrapping and feature randomness automatically.

In [ ]:
# Compare using scikit-learn's implementations
from sklearn.ensemble import BaggingRegressor

# Bagged trees using BaggingRegressor (all features)
bagging_model = BaggingRegressor(
    estimator=DecisionTreeRegressor(min_samples_split=10, min_samples_leaf=4),
    n_estimators=100,
    random_state=42
)
bagging_model.fit(X_train_reg, y_train_reg)
bagging_score = bagging_model.score(X_test_reg, y_test_reg)

# Random forest (feature randomness)
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_features=X_train_reg.shape[1] // 3,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42
)
rf_model.fit(X_train_reg, y_train_reg)
rf_score = rf_model.score(X_test_reg, y_test_reg)

print("Scikit-learn Comparison:")
print(f"Bagging (all features):     R² = {bagging_score:.4f}")
print(f"Random Forest (p/3 features): R² = {rf_score:.4f}")
print(f"\nRandom forest performs better by decorrelating the trees!")

### 🏃‍♂️ Try It Yourself

Experiment with feature randomness:
1. Try different `max_features` values (1, 3, 5, 10, None) and see how performance changes
2. Create a dataset where one feature is much stronger than others - does feature randomness help more?
3. Compare the predictions from bagged trees vs random forests - are they more different than you expected?

In [ ]:
# Your experimental code here


## Building Random Forest Models

Now that we understand how random forests work conceptually, let's build models for real problems. We'll use scikit-learn's `RandomForestClassifier` for classification and `RandomForestRegressor` for regression.

### Classification with Random Forests

Let's build a credit default classifier using a real dataset. We'll predict whether customers will default on their credit card payments.

In [ ]:
# Load the Default dataset from ISLP
try:
    from ISLP import load_data
    Default = load_data('Default')
    
    # Prepare features and target
    X_default = pd.get_dummies(Default[['balance', 'income', 'student']], drop_first=True)
    y_default = (Default['default'] == 'Yes').astype(int)
    
    print("✅ ISLP dataset loaded successfully!")
    print(f"Dataset size: {len(Default)} customers")
    print(f"Default rate: {y_default.mean():.1%}")
    print(f"\nFeatures: {list(X_default.columns)}")
    
except ImportError:
    print("⚠️ ISLP package not available. Using synthetic data instead.")
    print("To install ISLP: pip install ISLP")
    
    # Create synthetic default data
    np.random.seed(42)
    n_samples = 10000
    
    balance = np.random.gamma(2, 500, n_samples)
    income = np.random.normal(45000, 15000, n_samples)
    student = np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
    
    # Default probability increases with balance
    default_prob = 1 / (1 + np.exp(-(balance/1000 - 2)))
    y_default = (np.random.random(n_samples) < default_prob).astype(int)
    
    X_default = pd.DataFrame({
        'balance': balance,
        'income': income,
        'student_Yes': student
    })
    
    print(f"Synthetic dataset created: {len(X_default)} samples")
    print(f"Default rate: {y_default.mean():.1%}")

In [ ]:
# Split data
X_train_default, X_test_default, y_train_default, y_test_default = train_test_split(
    X_default, y_default, test_size=0.3, random_state=42, stratify=y_default
)

# Build random forest classifier
rf_classifier = RandomForestClassifier(
    n_estimators=100,      # Number of trees
    max_features='sqrt',   # √p features at each split (good for classification)
    random_state=42
)

rf_classifier.fit(X_train_default, y_train_default)

# Make predictions
y_pred_default = rf_classifier.predict(X_test_default)
y_pred_proba = rf_classifier.predict_proba(X_test_default)[:, 1]  # Probability of default

print("Random Forest Classifier trained successfully!")
print(f"\nModel contains {rf_classifier.n_estimators} trees")
print(f"Each tree considers {rf_classifier.max_features} features at each split")

In [ ]:
# Evaluate classification performance
from sklearn.metrics import accuracy_score

train_accuracy = rf_classifier.score(X_train_default, y_train_default)
test_accuracy = rf_classifier.score(X_test_default, y_test_default)

print("Classification Performance:")
print(f"Training accuracy: {train_accuracy:.3f}")
print(f"Test accuracy:     {test_accuracy:.3f}")
print(f"Overfitting gap:   {(train_accuracy - test_accuracy):.3f}")
print()
print("Confusion Matrix:")
cm = confusion_matrix(y_test_default, y_pred_default)
print(cm)
print()
print("Classification Report:")
print(classification_report(y_test_default, y_pred_default, 
                          target_names=['No Default', 'Default']))

**How Classification Predictions Work: Majority Voting**

Random forests make classification predictions through **majority voting**:
1. Each of the 100 trees independently predicts a class (0 or 1)
2. Count how many trees predict each class
3. The class with the most votes wins
4. Predicted probability = proportion of votes (e.g., 73/100 = 0.73)

In [ ]:
# Demonstrate voting mechanism for a single prediction
sample_idx = 0
sample_features = X_test_default.iloc[sample_idx:sample_idx+1]

# Get predictions from all individual trees
tree_predictions = []
for tree in rf_classifier.estimators_:
    pred = tree.predict(sample_features)[0]
    tree_predictions.append(pred)

# Count votes
votes_for_default = sum(tree_predictions)
votes_for_no_default = len(tree_predictions) - votes_for_default

print(f"Example prediction for sample {sample_idx}:")
print(f"  Votes for 'No Default' (0): {votes_for_no_default}")
print(f"  Votes for 'Default' (1):    {votes_for_default}")
print(f"  Final prediction: {rf_classifier.predict(sample_features)[0]}")
print(f"  Predicted probability of default: {votes_for_default/len(tree_predictions):.3f}")
print(f"  Actual label: {y_test_default.iloc[sample_idx]}")

### Regression with Random Forests

Now let's build a regression model to predict house prices using the Ames housing dataset.

In [ ]:
# Load Ames housing data
try:
    ames = pd.read_csv('https://raw.githubusercontent.com/bradleyboehmke/uc-bana-4080/main/data/ames_clean.csv')
    
    # Select numeric features
    numeric_features = ames.select_dtypes(include=[np.number]).columns.tolist()
    numeric_features.remove('SalePrice')
    
    X_ames = ames[numeric_features]
    y_ames = ames['SalePrice']
    
    print("✅ Ames housing data loaded successfully!")
    print(f"Dataset size: {len(ames)} homes")
    print(f"Number of features: {len(numeric_features)}")
    print(f"Price range: ${y_ames.min():,.0f} - ${y_ames.max():,.0f}")
    
except Exception as e:
    print(f"⚠️ Could not load Ames data: {e}")
    print("Using synthetic housing data instead.")
    
    # Create synthetic housing data
    np.random.seed(42)
    n_samples = 1000
    
    X_ames = pd.DataFrame({
        'GrLivArea': np.random.normal(1500, 500, n_samples),
        'OverallQual': np.random.randint(1, 11, n_samples),
        'YearBuilt': np.random.randint(1900, 2020, n_samples),
        'TotalBsmtSF': np.random.normal(1000, 400, n_samples),
        'GarageArea': np.random.normal(500, 200, n_samples)
    })
    
    # Price based on features
    y_ames = (100 * X_ames['GrLivArea'] + 
              10000 * X_ames['OverallQual'] + 
              50 * X_ames['YearBuilt'] +
              np.random.normal(0, 20000, n_samples))
    
    print(f"Synthetic dataset created: {len(X_ames)} samples")

In [ ]:
# Split data
X_train_ames, X_test_ames, y_train_ames, y_test_ames = train_test_split(
    X_ames, y_ames, test_size=0.3, random_state=42
)

# Build random forest regressor
rf_regressor = RandomForestRegressor(
    n_estimators=100,                    # Number of trees
    max_features=X_train_ames.shape[1]//3,  # p/3 features at each split (good for regression)
    random_state=42
)

rf_regressor.fit(X_train_ames, y_train_ames)

# Make predictions
y_pred_train = rf_regressor.predict(X_train_ames)
y_pred_test = rf_regressor.predict(X_test_ames)

print("Random Forest Regressor trained successfully!")
print(f"\nModel contains {rf_regressor.n_estimators} trees")
print(f"Each tree considers {rf_regressor.max_features} features at each split")

In [ ]:
# Evaluate regression performance
train_r2 = r2_score(y_train_ames, y_pred_train)
test_r2 = r2_score(y_test_ames, y_pred_test)
test_mae = mean_absolute_error(y_test_ames, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test_ames, y_pred_test))

print("Regression Performance:")
print(f"Training R²: {train_r2:.3f}")
print(f"Test R²:     {test_r2:.3f}")
print(f"Overfitting gap: {(train_r2 - test_r2):.3f}")
print()
print(f"Test MAE:  ${test_mae:,.0f}")
print(f"Test RMSE: ${test_rmse:,.0f}")
print(f"RMSE as % of mean price: {(test_rmse / y_test_ames.mean() * 100):.1f}%")

**How Regression Predictions Work: Averaging**

Random forests make regression predictions through **averaging**:
1. Each of the 100 trees independently predicts a numeric value
2. Calculate the mean: (tree₁ + tree₂ + ... + tree₁₀₀) / 100
3. This average becomes the final prediction

This averaging smooths out individual tree errors and produces more stable predictions!

In [ ]:
# Demonstrate averaging mechanism for a single prediction
sample_idx = 0
sample_features = X_test_ames.iloc[sample_idx:sample_idx+1]

# Get predictions from all individual trees
tree_predictions = []
for tree in rf_regressor.estimators_:
    pred = tree.predict(sample_features)[0]
    tree_predictions.append(pred)

print(f"Example prediction for house {sample_idx}:")
print(f"  Individual tree predictions range: ${min(tree_predictions):,.0f} - ${max(tree_predictions):,.0f}")
print(f"  Standard deviation of predictions: ${np.std(tree_predictions):,.0f}")
print(f"  Average (final prediction): ${np.mean(tree_predictions):,.0f}")
print(f"  Actual price: ${y_test_ames.iloc[sample_idx]:,.0f}")
print(f"  Prediction error: ${abs(np.mean(tree_predictions) - y_test_ames.iloc[sample_idx]):,.0f}")

### 🏃‍♂️ Try It Yourself

Experiment with random forest models:
1. **Classification**: Change `max_features` to different values ('sqrt', 'log2', None) - how does it affect accuracy?
2. **Regression**: Try different numbers of trees (10, 50, 100, 200) - plot how RMSE changes
3. **Compare**: Build a single decision tree and compare its performance to the random forest
4. **Prediction variance**: For the regression example, examine how much individual tree predictions vary for different houses

In [ ]:
# Your experimental code here


## Hyperparameter Tuning

Random forests have several important hyperparameters that control model complexity and performance. Let's explore the most critical ones and see how they impact results.

### Key Hyperparameters Overview

| Parameter | Purpose | Typical Values |
|-----------|---------|----------------|
| `n_estimators` | Number of trees | 100-500 |
| `max_features` | Features per split | 'sqrt' (classification), p/3 (regression) |
| `max_depth` | Maximum tree depth | None (no limit) or 10-20 |
| `min_samples_split` | Min samples to split node | 2-20 |
| `min_samples_leaf` | Min samples in leaf | 1-10 |

### 1. n_estimators: Number of Trees

More trees generally improve performance, but with diminishing returns. Let's visualize this.

In [ ]:
# Test different numbers of trees
tree_counts = [1, 5, 10, 25, 50, 100, 200, 300]
train_scores = []
test_scores = []

for n_trees in tree_counts:
    rf = RandomForestRegressor(n_estimators=n_trees, max_features='sqrt', random_state=42)
    rf.fit(X_train_ames, y_train_ames)
    
    train_scores.append(rf.score(X_train_ames, y_train_ames))
    test_scores.append(rf.score(X_test_ames, y_test_ames))

# Plot results
plt.figure(figsize=(8, 5))
plt.plot(tree_counts, train_scores, 'o-', label='Training R²', linewidth=2)
plt.plot(tree_counts, test_scores, 's-', label='Test R²', linewidth=2)
plt.xlabel('Number of Trees (n_estimators)', fontsize=11)
plt.ylabel('R² Score', fontsize=11)
plt.title('Impact of Number of Trees on Performance', fontsize=13, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Test R² with 10 trees:  {test_scores[2]:.4f}")
print(f"Test R² with 100 trees: {test_scores[5]:.4f}")
print(f"Test R² with 300 trees: {test_scores[7]:.4f}")
print(f"\nImprovement from 10 to 100: {((test_scores[5] - test_scores[2])/test_scores[2]*100):.1f}%")
print(f"Improvement from 100 to 300: {((test_scores[7] - test_scores[5])/test_scores[5]*100):.1f}%")
print("\n📊 Notice: Sharp improvement initially, then diminishing returns!")

### 2. max_features: Features Considered at Each Split

This parameter controls tree diversity! Lower values = more diverse trees = better averaging.

In [ ]:
# Test different max_features settings
n_features = X_train_ames.shape[1]
max_features_options = [1, int(np.sqrt(n_features)), n_features // 3, 
                       n_features // 2, n_features]
feature_labels = ['1', f'√p ({int(np.sqrt(n_features))})', 
                 f'p/3 ({n_features // 3})',
                 f'p/2 ({n_features // 2})', 
                 f'All ({n_features})']

train_scores = []
test_scores = []

for max_feat in max_features_options:
    rf = RandomForestRegressor(n_estimators=100, max_features=max_feat, random_state=42)
    rf.fit(X_train_ames, y_train_ames)
    
    train_scores.append(rf.score(X_train_ames, y_train_ames))
    test_scores.append(rf.score(X_test_ames, y_test_ames))

# Plot results
plt.figure(figsize=(8, 5))
x_pos = np.arange(len(max_features_options))
plt.plot(x_pos, train_scores, 'o-', label='Training R²', linewidth=2, markersize=8)
plt.plot(x_pos, test_scores, 's-', label='Test R²', linewidth=2, markersize=8)
plt.xticks(x_pos, feature_labels, rotation=15)
plt.xlabel('Features per Split (max_features)', fontsize=11)
plt.ylabel('R² Score', fontsize=11)
plt.title('Impact of Feature Randomness on Performance', fontsize=13, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

best_idx = np.argmax(test_scores)
print(f"Best test R² achieved with max_features = {feature_labels[best_idx]}")
print(f"Best R²: {test_scores[best_idx]:.4f}")
print("\n📊 Notice: Too few features = underfitting, all features = less diversity!")

### 3. max_depth: Maximum Tree Depth

Controls how deep individual trees can grow. Random forests can handle deeper trees than single trees because ensemble averaging reduces overfitting.

In [ ]:
# Test different maximum depths
depths = [3, 5, 10, 15, 20, None]  # None = no limit
depth_labels = ['3', '5', '10', '15', '20', 'No Limit']

train_scores = []
test_scores = []

for depth in depths:
    rf = RandomForestRegressor(n_estimators=100, max_depth=depth, 
                               max_features='sqrt', random_state=42)
    rf.fit(X_train_ames, y_train_ames)
    
    train_scores.append(rf.score(X_train_ames, y_train_ames))
    test_scores.append(rf.score(X_test_ames, y_test_ames))

# Plot results
plt.figure(figsize=(8, 5))
x_pos = np.arange(len(depths))
plt.plot(x_pos, train_scores, 'o-', label='Training R²', linewidth=2, markersize=8)
plt.plot(x_pos, test_scores, 's-', label='Test R²', linewidth=2, markersize=8)
plt.xticks(x_pos, depth_labels)
plt.xlabel('Maximum Tree Depth (max_depth)', fontsize=11)
plt.ylabel('R² Score', fontsize=11)
plt.title('Impact of Tree Depth on Performance', fontsize=13, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

best_idx = np.argmax(test_scores)
print(f"Best test R² with max_depth = {depth_labels[best_idx]}")
print(f"Best R²: {test_scores[best_idx]:.4f}")

### Comparing Default vs. Tuned Models

Let's compare a random forest with default parameters to one with carefully tuned hyperparameters.

In [ ]:
# Default random forest
rf_default = RandomForestRegressor(random_state=42)
rf_default.fit(X_train_ames, y_train_ames)

y_pred_default = rf_default.predict(X_test_ames)
default_r2 = r2_score(y_test_ames, y_pred_default)
default_rmse = np.sqrt(mean_squared_error(y_test_ames, y_pred_default))

print("Default Random Forest:")
print(f"  n_estimators: {rf_default.n_estimators}")
print(f"  max_features: {rf_default.max_features}")
print(f"  max_depth: {rf_default.max_depth}")
print(f"  Test R²: {default_r2:.4f}")
print(f"  Test RMSE: ${default_rmse:,.0f}")

In [ ]:
# Tuned random forest
rf_tuned = RandomForestRegressor(
    n_estimators=200,                      # More trees
    max_features=X_train_ames.shape[1]//3, # p/3 for regression
    max_depth=20,                          # Limit depth
    min_samples_split=5,                   # Require more samples to split
    min_samples_leaf=2,                    # Require more samples in leaves
    random_state=42
)
rf_tuned.fit(X_train_ames, y_train_ames)

y_pred_tuned = rf_tuned.predict(X_test_ames)
tuned_r2 = r2_score(y_test_ames, y_pred_tuned)
tuned_rmse = np.sqrt(mean_squared_error(y_test_ames, y_pred_tuned))

print("\nTuned Random Forest:")
print(f"  n_estimators: {rf_tuned.n_estimators}")
print(f"  max_features: {rf_tuned.max_features}")
print(f"  max_depth: {rf_tuned.max_depth}")
print(f"  Test R²: {tuned_r2:.4f}")
print(f"  Test RMSE: ${tuned_rmse:,.0f}")

In [ ]:
# Compare the two models
improvement_r2 = ((tuned_r2 - default_r2) / default_r2) * 100
improvement_rmse = ((default_rmse - tuned_rmse) / default_rmse) * 100

print("\n" + "="*50)
print("Performance Comparison")
print("="*50)
print(f"Default R²:  {default_r2:.4f}")
print(f"Tuned R²:    {tuned_r2:.4f}")
print(f"R² improvement: {improvement_r2:.1f}%")
print()
print(f"Default RMSE:  ${default_rmse:,.0f}")
print(f"Tuned RMSE:    ${tuned_rmse:,.0f}")
print(f"RMSE improvement: {improvement_rmse:.1f}%")
print("\n💡 Key Insight: Random forests work well out-of-the-box,")
print("   but tuning can squeeze out additional performance!")

### 🏃‍♂️ Try It Yourself

Experiment with hyperparameter tuning:
1. Test `min_samples_split` values (2, 5, 10, 20, 50) - how does it affect overfitting?
2. Try combining different parameters - what's your best combination?
3. Use the classification dataset (Default) and tune hyperparameters for best precision/recall balance
4. Create a small grid search: test all combinations of `n_estimators=[50, 100, 200]` and `max_depth=[10, 20, None]`

In [ ]:
# Your experimental code here


## End of Chapter Exercises

For these exercises, you'll compare decision trees to random forests across real business scenarios. Each exercise will help you understand how ensemble methods improve upon individual trees and how hyperparameter tuning can further enhance performance.

### Exercise 1: Baseball Salary Prediction (Regression)

**Company:** Professional baseball team  
**Goal:** Improve salary predictions by comparing individual decision trees to random forest ensembles  
**Dataset:** Hitters dataset from ISLP package

**Your Tasks:**

1. **Build three models and compare performance:**
   - **Model A:** `DecisionTreeRegressor` with `max_depth=4`
   - **Model B:** `RandomForestRegressor` with default parameters
   - **Model C:** `RandomForestRegressor` with tuned parameters (try `n_estimators=200`, `max_depth=6`, `min_samples_split=10`)

2. **Evaluate and compare:** Split data (70/30 train/test) and for each model calculate:
   - Training R² and test R²
   - Mean Absolute Error on test set
   - Root Mean Squared Error on test set
   - Overfitting gap (training R² - test R²)

3. **Performance analysis:**
   - Which model generalizes best to the test set?
   - How much improvement did the default random forest provide over the single tree?
   - Did tuning the hyperparameters further improve performance?

4. **Trade-offs:**
   - Compare model interpretability: Can you still extract clear business rules from the random forest?
   - Consider computational cost: How much longer did the random forest take to train?
   - Which model would you recommend to the team's management? Why?

5. **Business reflection:**
   - How confident would you be using each model for actual salary negotiations?
   - What are the risks of relying on the more complex random forest vs. the simpler tree?

In [ ]:
# Load and prepare the Hitters data
from ISLP import load_data

Hitters = load_data('Hitters')

# Remove missing salary values
Hitters_clean = Hitters.dropna(subset=['Salary'])

# Select numeric features for our models
features = ['Years', 'Hits', 'RBI', 'Walks', 'PutOuts']
X_hitters = Hitters_clean[features]
y_hitters = Hitters_clean['Salary']

print(f"Dataset size: {len(Hitters_clean)} players")
print(f"Features used: {features}")
print(f"\nFirst few rows:")
print(Hitters_clean[features + ['Salary']].head())

In [ ]:
# Your code for Exercise 1 here
# Build Model A: DecisionTreeRegressor


# Build Model B: RandomForestRegressor with default parameters


# Build Model C: RandomForestRegressor with tuned parameters


# Evaluate and compare all three models


### Exercise 2: Credit Default Classification

**Company:** Regional bank  
**Goal:** Improve default prediction accuracy while understanding the performance gains from ensemble methods  
**Dataset:** Default dataset from ISLP package

**Your Tasks:**

1. **Build and compare three classification models:**
   - **Model A:** `DecisionTreeClassifier` with `max_depth=3`, `min_samples_split=50`
   - **Model B:** `RandomForestClassifier` with default parameters
   - **Model C:** `RandomForestClassifier` with tuned parameters (try `n_estimators=200`, `max_depth=10`, `min_samples_split=20`, `max_features='sqrt'`)

2. **Evaluate on the imbalanced data:** Split data (70/30) and for each model examine:
   - Training vs. test accuracy
   - Precision and recall for the "default" class specifically
   - Confusion matrix on test set
   - Which model has the best balance of precision and recall?

3. **Understand the ensemble advantage:**
   - Calculate the overfitting gap for each model
   - How did random forests reduce overfitting compared to the single tree?
   - Did the tuned random forest improve further on precision/recall for defaults?

4. **Feature importance (Random Forest only):**
   - Use `.feature_importances_` on your best random forest model
   - Which feature is most important for predicting defaults?
   - How does this align with business intuition?

5. **Business application:**
   - The bank loses $10,000 on each default but gains $500 in interest from good customers
   - Using the confusion matrices, calculate the expected profit/loss for each model
   - Which model would you deploy in production? Why?
   - What threshold adjustments might you make given the cost structure?

In [ ]:
# Load and prepare the Default dataset
Default = load_data('Default')

# Prepare features - encode student as binary
Default_encoded = pd.get_dummies(Default, columns=['student'], drop_first=True)
Default_encoded['default_binary'] = (Default_encoded['default'] == 'Yes').astype(int)

# Select features
X_default = Default_encoded[['balance', 'income', 'student_Yes']]
y_default = Default_encoded['default_binary']

print(f"Dataset size: {len(Default)} customers")
print(f"Default rate: {y_default.mean():.1%}")
print(f"\nFeatures prepared:")
print(X_default.head())

In [ ]:
# Your code for Exercise 2 here
# Build Model A: DecisionTreeClassifier


# Build Model B: RandomForestClassifier with default parameters


# Build Model C: RandomForestClassifier with tuned parameters


# Evaluate and compare all three models


# Calculate feature importance for best model


# Calculate expected profit/loss for each model


### Exercise 3: Stock Market Direction Prediction (Optional Challenge)

**Company:** Investment management firm  
**Goal:** Test whether random forests can capture market patterns better than single decision trees  
**Dataset:** Weekly dataset from ISLP package

**Your Tasks:**

1. **Build three models for market prediction:**
   - **Model A:** `DecisionTreeClassifier` with `max_depth=3`
   - **Model B:** `RandomForestClassifier` with default parameters
   - **Model C:** `RandomForestClassifier` with tuned parameters of your choice

2. **Time-series evaluation:**
   - Split data chronologically (first 80% train, last 20% test)
   - Calculate test accuracy for each model
   - Compare to baseline: what's the accuracy of always predicting "Up"?

3. **Performance comparison:**
   - Did random forests improve over the single tree?
   - Are any of the models beating the baseline?
   - What does this tell you about market predictability?

4. **Feature importance analysis:**
   - Examine feature importance from your best random forest
   - Which lag periods matter most?
   - Do these patterns make financial sense?

5. **Challenge questions:**
   - Why might even random forests struggle with financial market prediction?
   - What characteristics of stock market data make it fundamentally difficult for tree-based methods?
   - Would you recommend using any of these models for actual trading? What would be the risks?
   - How might you modify the approach to make it more suitable for financial prediction?

In [ ]:
# Load and prepare the Weekly stock market data
Weekly = load_data('Weekly')

# Prepare features and target
Weekly_encoded = Weekly.copy()
Weekly_encoded['Direction_binary'] = (Weekly_encoded['Direction'] == 'Up').astype(int)

# Use lag variables as features
lag_features = ['Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5']
X_weekly = Weekly_encoded[lag_features]
y_weekly = Weekly_encoded['Direction_binary']

print(f"Dataset size: {len(Weekly)} weeks")
print(f"Up weeks: {y_weekly.mean():.1%}")
print(f"\nLag features:")
print(X_weekly.head())

In [ ]:
# Your code for Exercise 3 here
# Build Model A: DecisionTreeClassifier


# Build Model B: RandomForestClassifier with default parameters


# Build Model C: RandomForestClassifier with tuned parameters


# Perform time-series split and evaluation


# Calculate baseline accuracy


# Feature importance analysis


# Answer challenge questions in markdown cell below


---

## 📝 Chapter Summary

Congratulations on completing the Random Forests chapter! Here's what you've learned:

### 🎯 Core Concepts

**Random forests transform the instability of individual decision trees into a strength** by combining hundreds of trees trained on different data samples. The key insight: while any single tree might overfit or make errors, the collective wisdom of diverse trees averages out mistakes, producing accurate and stable predictions.

### 🔑 Key Mechanisms

Random forests create diversity through two mechanisms:

1. **Bootstrap Aggregating (Bagging)**: Each tree trains on a random sample with replacement
2. **Feature Randomness**: Each split considers only a random subset of features
   - Typically √p for classification
   - Typically p/3 for regression

These mechanisms decorrelate trees and prevent dominant features from controlling the forest.

### 🤖 How Predictions Work

- **Classification**: Majority voting across all trees
  - Each tree votes for a class
  - The class with most votes wins
  - Predicted probability = proportion of votes
  
- **Regression**: Averaging across all trees
  - Each tree predicts a numeric value
  - Average becomes the final prediction
  - Smooths out individual tree errors

### ⚙️ Key Hyperparameters

| Parameter | Purpose | Typical Values |
|-----------|---------|----------------|
| `n_estimators` | Number of trees | 100-500 |
| `max_features` | Features per split | 'sqrt' (classification), p/3 (regression) |
| `max_depth` | Maximum tree depth | None (no limit) or 10-20 |
| `min_samples_split` | Min samples to split | 2-20 |
| `min_samples_leaf` | Min samples in leaf | 1-10 |

### 💡 Important Insights

- **Out-of-the-box effectiveness**: Random forests work remarkably well with default settings, requiring minimal tuning
- **Robust to overfitting**: Ensemble averaging makes them more stable than single trees
- **Business applications**: Credit risk, fraud detection, customer churn, predictive maintenance, and more
- **Trade-offs**: Better performance but less interpretable than single trees

### 🚀 Next Steps

In the next chapter, you'll explore **feature importance**—a built-in mechanism for quantifying each feature's contribution to predictions and identifying the key drivers behind your outcomes.

---

### 🎓 Key Takeaways for Practice

1. **Start with defaults**: Random forests perform well without extensive tuning
2. **More trees ≈ better performance**: But with diminishing returns after 100-200 trees
3. **Feature randomness matters**: Don't skip `max_features` tuning for best results
4. **Balance interpretability vs. accuracy**: Consider your stakeholders when choosing between single trees and forests
5. **Use for tabular data**: Random forests excel with structured, tabular datasets

**Great work!** You now have a solid understanding of one of the most powerful and widely-used machine learning algorithms in practice. 🎉